In [ ]:
from pathlib import Path

DRP_VERSION = "w_2025_06"
COLLECTION_TAG = "DM-48810"

raw_dir = Path(f"/sdf/data/rubin/shared/lsdb_commissioning") / "raw_delucchi" / DRP_VERSION

file_pointer = raw_dir / "paths" / f"sourceTable.txt"
with file_pointer.open("r", encoding="utf8") as _text_file:
    paths = _text_file.readlines()
paths = [path.strip() for path in paths]
paths[:10]

In [ ]:
import pyarrow.parquet as pq

from hats.pixel_math.healpix_shim import radec2pix
failed_files = 0
for path in paths:
    parquet_file = pq.ParquetFile(path)
    
    ra_min = parquet_file.metadata.row_group(0).column(7).statistics.min
    ra_max = parquet_file.metadata.row_group(0).column(7).statistics.max
    dec_min = parquet_file.metadata.row_group(0).column(8).statistics.min
    dec_max = parquet_file.metadata.row_group(0).column(8).statistics.max

    try:
        _ = radec2pix(10, [ra_min, ra_min, ra_max, ra_max], [dec_min, dec_max, dec_min, dec_max])
    except:
        print(path)
        print([ra_min, ra_max, dec_min, dec_max])
        break
        failed_files += 1

failed_files